# 1. Basic RAG 구축 실습

이 노트북에서는 LangChain을 사용하여 기본적인 RAG(Retrieval-Augmented Generation) 파이프라인을 구축합니다.
한국은행 업종 보고서를 데이터로 사용하여 질문에 답변하는 봇을 만듭니다.

**목표:**
1. PDF 문서를 로드하고 청크(Chunk)로 분할합니다.
2. Vector DB(Chroma)에 문서를 임베딩하여 저장합니다.
3. **Custom Tool**을 정의하여 Agent가 능동적으로 검색을 수행하도록 만듭니다.
4. 현재 구현의 **문제점**을 확인합니다.

이제 환경 변수를 등록해줍시다.

- 서비스를 활용하기 위해 API KEY 값이 필요합니다. 지정된 환경변수에 API KEY를 저장하면 쉽게 API를 연동할 수 있습니다.
- .env에 환경변수를 저장하고 한 번에 불러옵니다.
- .env 파일은 기본적으로 숨김파일이므로, colab에서 볼 때는 아래와 같이 눈 표시(파란색 원)를 체크해서 파일을 볼 수 있습니다.

In [ ]:
import sys
import os
from dotenv import load_dotenv

# 1. 현재 작업 디렉토리 설정
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "llmops_class"))

# 2. sys.path에 추가 (이제 Python이 'app' 폴더를 찾을 수 있습니다)
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"✅ Project Root added to sys.path: {project_root}")

# 3. .env 파일 로드
dotenv_path = os.path.join("llmops_class", ".env")

if os.path.exists(dotenv_path):
    print(f"Loading .env from: {dotenv_path}")
    load_dotenv(dotenv_path)


In [2]:
# API Key 확인 (선택 사항)
if "OPENAI_API_KEY" not in os.environ:
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

## LangChain Review

## LangChain Chat LLM (Chat completion LLM) 사용하기

Chat LLM은 메시지 리스트를 입력 받아 응답을 생성하는 대화형 LLM입니다. 챗봇, 대화형 워크플로우, 대화형 RAG, 에이전트 개발 등에 적용되는 가장 보편적인 모델입니다. 데이터를 처리하려면 invoke 메서드에 메시지 목록을 전달하면 됩니다.

LangChain [Chat llm](https://python.langchain.com/docs/integrations/chat/) 객체는 다음과 같이 생성합니다.

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o-mini", model_provider="openai", temperature=0.6)

- model : 사용할 llm 모델 명칭
- model_provider : `openai`, `anthropic`, `google_genai` 등. pip install로 [제품사 패키지 추가설치 필요](https://python.langchain.com/docs/integrations/chat/#featured-providers).
    - 예를 들어 openai의 경우 langchain-openai 설치
- temperature : 낮을 수록 일관되고 높을 수록 무작위적이고 창의적이 됩니다.

.invoke 메서드로 LLM에 프롬프트 텍스트를 전달합니다. 결과는 AIMessage 타입의 데이터입니다.

In [6]:
ai_response = llm.invoke("안녕하세요? 당신은 어떤 모델인가요?")

(1) Response 내용 : 응답 텍스트는 content 속성에 있습니다.

In [ ]:
ai_response.content

(2) 응답 관련 메타데이터도 확인할 수 있습니다.

llm provider의 api platform에서 지원 모델과 토큰 당 가격을 확인 가능 : [openai](https://platform.openai.com/docs/pricing)

In [ ]:
ai_response.response_metadata

(3) langchain llm의 response 데이터 타입은 Message의 한 종류인 `AIMessage` 입니다.

In [ ]:
type(ai_response)

In [ ]:
ai_response

### LLM과 메시지 리스트

우리가 사용하는 ChatLLM은 대화형 데이터를 처리하기 위해 SystemMessage, HumanMessage, AIMessage, ToolMessage 객체를 사용하여 [메시지](https://python.langchain.com/docs/concepts/messages/)를 정의합니다.

In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

messages = [
    SystemMessage("당신은 오직 한국어로 답변하는 친절한 AI입니다. 간결하게 답변합니다."),
    HumanMessage("Explain RAG(Retrieval-Augmented Generation.)"),
]

llm.invoke(messages)

### LangChain 에이전트

LangGraph **Agent**는 주어진 작업을 수행하기 위해 연동된 외부 도구를 호출하여 문제를 해결합니다.

LLM(언어 모델)을 기반으로 하며, 그래프 내에서 **도구(Tool or Functions)**와 상호작용하여 LLM에게 어려운 작업을 처리할 수 있습니다. LLM은 자연어 처리 능력만으로 제한적일 수 있으므로, 특정 작업(예: 데이터 검색, 계산, 파일 읽기 등)을 수행하기 위해 외부의 도구를 호출합니다.


```
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from datetime import date

# 오늘 날짜를 YYYY-MM-DD 형식으로 가져옵니다.
today_date = date.today().strftime("%Y-%m-%d")

# Agent 생성 (User-requested style)

tools = file_management_tools
system_prompt = f"""당신은 친절하게 답변하는 대화형 에이전트입니다.
                    유저의 요청을 처리하기 위해 필요 시 Tool을 호출해 사용할 수 있습니다.
                    유저가 시점을 밝히지 않았다면 현재 시점 기준으로 검색하세요.
                    오늘의 날짜 : {today_date} """


test_agent = create_agent("openai:gpt-4o", tools=tools, system_prompt=system_prompt)

```

## RAG Agent 구축하기

### 1. 데이터 로드 및 전처리

이제 RAG 구축을 위한 데이터를 읽어옵니다.

- 한국은행에제 RAG 구축을 위한 데이터를 읽어옵니다.

- 한국은행 [주력산업 모니터링 보고서](https://www.bok.or.kr/portal/singl/newsData/list.do?pageIndex=&targetDepth=3&menuNo=201127&syncMenuChekKey=1&depthSubMain=&subMainAt=&searchCnd=1&searchKwd=&depth2=201042&depth3=201127&date=&sdate=&edate=&sort=1&pageUnit=10)입니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 데이터 경로 설정
DATA_DIR = "llmops_class/data/bok_major_industry_reports"

# 모든 PDF 파일 경로 찾기
pdf_files = glob.glob(os.path.join(DATA_DIR, "*.pdf"))
print(f"Found {len(pdf_files)} PDF files: {pdf_files}")

docs = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())

print(f"Total pages loaded: {len(docs)}")

In [ ]:
# 문서 분할 (Chunking)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(docs)
print(f"Created {len(docs)} chunks.")

In [ ]:
# 데이터 확인
from IPython.display import display, Markdown

print("--- Sample Chunks ---")
for doc in docs[10:15]:
    display(Markdown(doc.page_content))
    print("=====================================\n\n")

### 2. Vector DB 구축 (Chroma)

임베딩 모델로는 `text-embedding-3-large`를 사용하고, Vector DB는 `Chroma`를 사용합니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

# 벡터 저장소 생성 (메모리 내)
report_vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    # persist_directory="llmops_class/data/chroma_db", # 이것을 주석처리하면, 테스트를 위해 Inmemory로 생성합니다. 이미 해당 경로에 vectorsotre가 구축되어있습니다.
    collection_name="basic_rag"
)

report_retriever = report_vectorstore.as_retriever()

# 검색 테스트
query = "2024년 4분기 반도체 업황"
retrieved_docs = report_retriever.invoke(query)

print(f"-- Search Results for '{query}' --")
for doc in retrieved_docs:
    display(Markdown(str(doc.metadata)))
    print(doc.page_content[:200] + "...")
    print("=====================================\n\n")

### 3. Agent Tool (Custom Tool Definition)

여기서는 `create_retriever_tool` 대신 **`@tool` 데코레이터**를 사용하여 커스텀 툴을 정의합니다.
이렇게 하면:
1. `limit` 등을 인자로 받아 LLM이 검색 개수를 조절할 수 있습니다.
2. 검색 결과 포맷팅(Metadata 포함)이나 로깅을 우리가 원하는 대로 제어할 수 있습니다.

In [21]:
from langchain_core.tools import tool

@tool(parse_docstring=True)
def search_bok_reports(query: str, limit: int = 4) -> str:
    """Search the Bank of Korea industry reports for relevant information.

    Args:
        query: The search query to find relevant information in the reports.
        limit: The maximum number of documents to retrieve. Defaults to 4.
    """
    print(f"[Tool Log] Searching for '{query}' with limit={limit}...")

    # Retriever의 'k' 값을 동적으로 조절하기 위해 search_kwargs 업데이트
    report_retriever = report_vectorstore.as_retriever()
    docs = report_retriever.invoke(query, k=limit)

    result_text = ""
    for i, doc in enumerate(docs):
        # Metadata 정보도 답변 생성에 참고할 수 있도록 포함
        meta_str = str(doc.metadata)
        result_text += f"[Document {i+1}]\nMetadata: {meta_str}\nContent: {doc.page_content}\n\n"

    return result_text

tools = [search_bok_reports]

In [ ]:
# Custom Tool 테스트 (Direct Invocation)
# 에이전트 연결 전, 도구가 잘 작동하는지 단독으로 테스트합니다.
test_query = "반도체 수출 전망"
print(f"Testing tool with query: {test_query}\n")

tool_output = search_bok_reports.invoke({"query": test_query, "limit": 2})
print(tool_output)

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Agent 생성 (User-requested style)
# Note: 'create_agent' signature depends on the specific package version/wrapper user has.
rag_agent = create_agent("openai:gpt-4o", tools=tools)

# Agent 테스트
input_query = "24년도 2분기 반도체 업황을 핵심 위주로 요약해줘."

response = rag_agent.invoke({"messages": [("user", input_query)]})

print(f"Q: {input_query}\n")
print(f"A: {response['messages'][-1].content}")

### 4. Self-Querying Retriever (셀프 쿼리 리트리버)로 성능 개선하기

Self-Querying Retriever는 사용자의 **자연어 질문을 LLM이 직접 분석**하여, 검색에 필요한 **의미 검색용 쿼리 문자열**과 **메타데이터 필터 조건**을 **자동으로 생성**하고 이를 사용하여 벡터 저장소를 검색하는 지능적인 RAG 기법입니다. 사용자가 복잡한 필터 구문을 알 필요 없이, 일상적인 대화처럼 질문해도 정교한 필터링이 가능해집니다.

In [35]:
import re   
import os
from llmops_class.app.utils.pymupdf4llm_loader import PyMuPDF4LLMLoader

docs = []
for file_path in pdf_files:
    filename = os.path.basename(file_path)
    # 예: "2024_1사분기_..."
    match = re.search(r"(\d{4})_(\d)사분기", filename ) # 메타데이터 추출
    year = int(match.group(1)) if match else 2024
    quarter = int(match.group(2)) if match else 0
    print(f"Loading {filename} (Year={year}, Quarter={quarter})...")
    
    loader = PyMuPDF4LLMLoader(file_path, mode="page", extract_images=False)
    file_docs = loader.load()

    for doc in file_docs:
        doc.metadata["year"] = year
        doc.metadata["quarter"] = quarter
        doc.metadata["source"] = filename

    docs.extend(file_docs)

Loading 2024_4사분기_주력산업_모니터링_보고서.pdf (Year=2024, Quarter=4)...
   🚀 39개 페이지 병렬 분석 시작...
   ⏳ [Progress] 1/39 (2.6%) - Page 1 완료
   ⏳ [Progress] 2/39 (5.1%) - Page 2 완료
   ⏳ [Progress] 3/39 (7.7%) - Page 3 완료
   ⏳ [Progress] 4/39 (10.3%) - Page 4 완료
   ⏳ [Progress] 5/39 (12.8%) - Page 5 완료
   ⏳ [Progress] 6/39 (15.4%) - Page 6 완료
   ⏳ [Progress] 7/39 (17.9%) - Page 7 완료
   ⏳ [Progress] 8/39 (20.5%) - Page 8 완료
   ⏳ [Progress] 9/39 (23.1%) - Page 9 완료
   ⏳ [Progress] 10/39 (25.6%) - Page 10 완료
   ⏳ [Progress] 11/39 (28.2%) - Page 11 완료
   ⏳ [Progress] 12/39 (30.8%) - Page 12 완료
   ⏳ [Progress] 13/39 (33.3%) - Page 13 완료
   ⏳ [Progress] 14/39 (35.9%) - Page 14 완료
   ⏳ [Progress] 15/39 (38.5%) - Page 15 완료
   ⏳ [Progress] 16/39 (41.0%) - Page 16 완료
   ⏳ [Progress] 17/39 (43.6%) - Page 17 완료
   ⏳ [Progress] 18/39 (46.2%) - Page 18 완료
   ⏳ [Progress] 19/39 (48.7%) - Page 19 완료
   ⏳ [Progress] 20/39 (51.3%) - Page 20 완료
   ⏳ [Progress] 21/39 (53.8%) - Page 21 완료
   ⏳ [Progress] 22/39 (56.4%) -

In [36]:
from langchain_text_splitters import MarkdownTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.retrievers.self_query.base import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo

# 1. 텍스트 분할 (마크다운 헤더 구조 고려)
text_splitter = MarkdownTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 2. 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    # persist_directory="llmops_class/data/chroma_db", # 이것을 주석처리하면, 테스트를 위해 Inmemory로 생성합니다. 이미 해당 경로에 vectorsotre가 구축되어있습니다.
    collection_name="self_query"
)
print("Vectorstore 구축 완료!")


# 3. 메타데이터 필드 정보
metadata_field_info = [
    AttributeInfo(
        name="year",
        description="해당 보고서의 연도를 나타내는 정수 값 (예: 2024)",
        type="integer",
    ),
    AttributeInfo(
        name="quarter",
        description="해당 보고서의 분기를 나타내는 숫자로 1, 2, 3, 4 중 하나.",
        type="integer",
    ),
]
document_content_description = "Bank of Korea Industry Reports"

# 4. SelfQueryRetriever 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0).with_config({"tags": ["exclude_from_stream"]})

sq_retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=False
)

Vectorstore 구축 완료!


In [42]:
print("24년도 2분기 조선업 업황")
docs = sq_retriever.invoke("24년도 2분기 조선업 업황")
for doc in docs:
    display(Markdown(str(f"#{doc.metadata['year']}년, {doc.metadata['quarter']}분기")))
    print("=====================================")
print("24년도 4분기 반도체 산업 동향")
docs = sq_retriever.invoke("24년도 4분기 반도체 산업 동향")
for doc in docs:
    display(Markdown(str(f"#{doc.metadata['year']}년, {doc.metadata['quarter']}분기")))
    print("=====================================")
print("24년도 2분기 이후의 핸드폰 관련 자료")
docs = sq_retriever.invoke("24년도 2분기 이후의 핸드폰 관련 자료")
for doc in docs:
    display(Markdown(str(f"#{doc.metadata['year']}년, {doc.metadata['quarter']}분기")))
    print("=====================================")


24년도 2분기 조선업 업황


#2024년, 2분기

#2024년, 2분기

#2024년, 2분기

#2024년, 2분기

...

24년도 2분기 이후의 핸드폰 관련 자료


#2024년, 3분기

#2024년, 4분기

#2024년, 3분기

#2024년, 3분기

### 5. Multimodal Retriever (멀티모달 리트리버)

이제 Self-Querying Retriever에 추가적으로 이미지와 텍스트를 동시에 검색할 수 있는 리트리버를 만들어보겠습니다. 


In [44]:
import re   
import os
from llmops_class.app.utils.pymupdf4llm_loader import PyMuPDF4LLMLoader
from langchain_google_genai import ChatGoogleGenerativeAI

docs = []
for file_path in pdf_files:
    filename = os.path.basename(file_path)
    # 예: "2024_1사분기_..."
    match = re.search(r"(\d{4})_(\d)사분기", filename ) # 메타데이터 추출
    year = int(match.group(1)) if match else 2024
    quarter = int(match.group(2)) if match else 0
    print(f"Loading {filename} (Year={year}, Quarter={quarter})...")
    
    loader = PyMuPDF4LLMLoader(file_path, mode="page", 
                                extract_images=True, 
                                image_output_dir="./extracted_images",
                                model=ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0))
    file_docs = loader.load()

    for doc in file_docs:
        doc.metadata["year"] = year
        doc.metadata["quarter"] = quarter
        doc.metadata["source"] = filename

    docs.extend(file_docs)

Loading 2024_4사분기_주력산업_모니터링_보고서.pdf (Year=2024, Quarter=4)...
📂 [Loader] PDF 로드: llmops_class/data/bok_major_industry_reports/2024_4사분기_주력산업_모니터링_보고서.pdf (Images: Analysis (Parallel x5), Mode: page)
   🚀 39개 페이지 병렬 분석 시작...
   ⏳ [Progress] 1/39 (2.6%) - Page 1 완료
   ⏳ [Progress] 2/39 (5.1%) - Page 2 완료
   ⏳ [Progress] 3/39 (7.7%) - Page 3 완료
   ⏳ [Progress] 4/39 (10.3%) - Page 4 완료
   ⏳ [Progress] 5/39 (12.8%) - Page 5 완료
   ⏳ [Progress] 6/39 (15.4%) - Page 6 완료
   ⏳ [Progress] 7/39 (17.9%) - Page 7 완료
   ⏳ [Progress] 8/39 (20.5%) - Page 8 완료
   ⏳ [Progress] 9/39 (23.1%) - Page 9 완료
   ⏳ [Progress] 10/39 (25.6%) - Page 10 완료
   ⏳ [Progress] 11/39 (28.2%) - Page 11 완료
   ⏳ [Progress] 12/39 (30.8%) - Page 12 완료
   ⏳ [Progress] 13/39 (33.3%) - Page 13 완료
   ⏳ [Progress] 14/39 (35.9%) - Page 14 완료
   ⏳ [Progress] 15/39 (38.5%) - Page 15 완료
   ⏳ [Progress] 16/39 (41.0%) - Page 16 완료
   ⏳ [Progress] 17/39 (43.6%) - Page 17 완료
   ⏳ [Progress] 18/39 (46.2%) - Page 18 완료
   ⏳ [Progress] 19/39 (

In [ ]:
from langchain_text_splitters import MarkdownTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.retrievers.self_query.base import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo

# 1. 텍스트 분할 (마크다운 헤더 구조 고려)
text_splitter = MarkdownTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 2. 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    # persist_directory="llmops_class/data/chroma_db", # 이것을 주석처리하면, 테스트를 위해 Inmemory로 생성합니다. 이미 해당 경로에 vectorsotre가 구축되어있습니다.
    collection_name="multimodal"
)
print("Vectorstore 구축 완료!")

metadata_field_info = [
    AttributeInfo(
        name="year",
        description="해당 보고서의 연도를 나타내는 정수 값 (예: 2024)",
        type="integer",
    ),
    AttributeInfo(
        name="quarter",
        description="해당 보고서의 분기를 나타내는 숫자로 1, 2, 3, 4 중 하나.",
        type="integer",
    ),
]
document_content_description = "Bank of Korea Industry Reports"

llm = ChatOpenAI(model="gpt-4o", temperature=0).with_config({"tags": ["exclude_from_stream"]})

multimodal_retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=False
)

테스트

In [ ]:
retrieved_docs = multimodal_retriever.invoke("24년도 1분기 반도체 수출의 증가폭 관련 그래프를 설명해줘")

In [107]:
retrieved_docs

[Document(id='76712277-ea7d-4383-935c-6d2d087d3cb5', metadata={'creator': 'Hwp 2018 10.0.0.10640', 'format': 'PDF 1.4', 'page': 5, 'file_path': 'llmops_class/data/bok_major_industry_reports/2024_1사분기_주력산업_모니터링_보고서.pdf', 'producer': 'Hancom PDF 1.3.0.538', 'quarter': 1, 'title': '', 'has_images': True, 'page_count': 38, 'year': 2024, 'modDate': "D:20240514181534+09'00'", 'keywords': '', 'subject': '', 'creationDate': "D:20240514181534+09'00'", 'author': '0510190', 'source': '2024_1사분기_주력산업_모니터링_보고서.pdf', 'trapped': ''}, page_content='![](./extracted_images/2024_1사분기_주력산업_모니터링_보고서.pdf-4-30.png)\n\n> **[이미지 설명]** Here is a detailed description of the image for RAG context retrieval:\n\n**Overall Description:**\n\nThe image is a line and bar graph comparing semiconductor export trends with overall export growth rates. The graph spans from 2018.1 to 2024.1. It uses a dual y-axis to represent different units: percentage change year-over-year (%) on the left and billions of US dollars on the 

In [ ]:
from pathlib import Path

BASE_DIR = Path("./llmops_class/") # notebook이 보고 있는 base dir입니다.
DATA_DIR = BASE_DIR / "data" / "extracted_images" # 이미지가 저장된 dir입니다.

combined_text = []
image_paths = []

# 마크다운 이미지 링크 패턴: ![alt](path)
img_pattern = re.compile(r'!\[.*?\]\((.*?)\)')

for image_doc in retrieved_docs:
    content = image_doc.page_content

    # A. 이미지 경로 추출
    found_paths = img_pattern.findall(content)

    for path in found_paths:

        filename = os.path.basename(path) # 파일명 추출
        abs_path = DATA_DIR / filename # 절대 경로 생성 (Agent가 파일을 열 때 사용)

        # 중복 제거 및 실제 파일 존재 여부 확인 (안전장치)
        if path not in image_paths:
            if abs_path.exists():
                image_paths.append(str(abs_path))

    # B. 텍스트 누적 (페이지 정보 포함하여 LLM이 출처를 알 수 있게 함)
    page_num = image_doc.metadata.get('page', 'Unknown')
    combined_text.append(f"--- [Page {page_num}] ---\n{content}")


In [104]:
found_paths

['./extracted_images/2024_1사분기_주력산업_모니터링_보고서.pdf-26-95.png',
 './extracted_images/2024_1사분기_주력산업_모니터링_보고서.pdf-26-96.png']

In [105]:
{
    "context": "\n\n".join(combined_text), # 검색된 텍스트 전체
    "images": image_paths,                 # 추출된 이미지 경로 리스트
    "source": "한국은행 8대 업종 모니터링 보고서" # 출처 태그
}

{'context': '--- [Page 5] ---\n**Textual Elements:**\n\n*   **(전년동기대비, %):**  Indicates "Year-over-year change, %" (used for the left y-axis).\n*   **(억달러):** Indicates "Billions of US Dollars" (used for the right y-axis).\n*   **반도체 수출액(우축):**  Indicates "Semiconductor Exports (Right Axis)" - refers to the green bars.\n*   **반도체 수출증가율(좌축):** Indicates "Semiconductor Export Growth Rate (Left Axis)" - refers to the red line.\n*   **전체 수출증가율(좌축):** Indicates "Overall Export Growth Rate (Left Axis)" - refers to the blue line.\n\n**In summary, the image presents a comparative analysis of semiconductor export performance against overall export growth, highlighting trends and fluctuations over a specific time period.**\n\n\n![](./extracted_images/2024_1사분기_주력산업_모니터링_보고서.pdf-4-31.png)\n\n> **[이미지 설명]** Here is a detailed description of the image for RAG context retrieval:\n\nThe image is a line and bar graph showing the percentage change compared to the same period of the previous year. The y